## Importing the modules

In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Load the dataset
df = pd.read_csv("abroad  - Sheet1.csv")

## Data preprocessing

In [3]:
# Step 2: Convert non-numeric values in "FEES" to NaN and handle missing values
df["FEES"] = pd.to_numeric(df["FEES"], errors='coerce')
df["FEES"].fillna(df["FEES"].median(), inplace=True)

C:\Users\TSA\AppData\Local\Temp\ipykernel_29692\3018806989.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["FEES"].fillna(df["FEES"].median(), inplace=True)


In [4]:
# Step 3: Apply Label Encoding to categorical columns
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['COUNTRY'] = label_encoder.fit_transform(df['COUNTRY'])
df['COURSE TYPE'] = label_encoder.fit_transform(df['COURSE TYPE'])
df['COURSE (SPECIALIZATION)'] = label_encoder.fit_transform(df['COURSE (SPECIALIZATION)'])

In [5]:
# Step 4: Prepare features and target variable
X = df[['COUNTRY', 'COURSE TYPE', 'COURSE (SPECIALIZATION)']]  # Features
y = df['FEES']  # Target

## Training the model

In [6]:
# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
# Define the objective function for Hyperopt
def objective(params):
    gbm = lgb.LGBMRegressor(
        objective='regression',
        metric='rmse',
        boosting_type='gbdt',
        num_leaves=max(int(params['num_leaves']), 2),  # Ensure num_leaves is at least 2
        learning_rate=params['learning_rate'],
        feature_fraction=params['feature_fraction'],
        bagging_fraction=params['bagging_fraction'],
        bagging_freq=int(params['bagging_freq']),
        max_depth=int(params['max_depth']),
        n_estimators=100
    )
    
    gbm.fit(X_train, y_train)
    y_pred = gbm.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    return {'loss': rmse, 'status': STATUS_OK}


In [8]:
# Define the search space for hyperparameters
space = {
    'num_leaves': hp.quniform('num_leaves', 2, 256, 1),  # Ensure minimum value is 2
    'learning_rate': hp.loguniform('learning_rate', -5, -1),
    'feature_fraction': hp.uniform('feature_fraction', 0.5, 1.0),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 10, 1),
    'max_depth': hp.quniform('max_depth', 3, 20, 1)
}

In [9]:
# Optimize the hyperparameters using Hyperopt
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

[LightGBM] [Warning] feature_fraction is set=0.8829440116409468, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8829440116409468
[LightGBM] [Warning] bagging_fraction is set=0.7354297048177948, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7354297048177948
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3            
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines                                        
[LightGBM] [Warning] feature_fraction is set=0.8829440116409468, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8829440116409468
[LightGBM] [Warning] bagging_fraction is set=0.7354297048177948, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7354297048177948
[LightGBM] [Warning] bagging_freq is set=3, subsample_freq=0 will be ignored. Current value: bagging_freq=3            
[LightGBM] [Info] Auto-choosing row-wi

C:\Users\TSA\anaconda3\envs\tf\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(

  File "C:\Users\TSA\anaconda3\envs\tf\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(

  File "C:\Users\TSA\anaconda3\envs\tf\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:

  File "C:\Users\TSA\anaconda3\envs\tf\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,

  File "C:\Users\TSA\anaconda3\envs\tf\lib\subprocess.py", line 1436, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits w

C:\Users\TSA\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits w

...

C:\Users\TSA\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf           

C:\Users\TSA\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf                                             
[LightGBM] [Warning] No further splits w

C:\Users\TSA\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [10]:
# Print the best hyperparameters
print("Best Hyperparameters:", best)

Best Hyperparameters: {'bagging_fraction': 0.7553040682932186, 'bagging_freq': 2.0, 'feature_fraction': 0.8660324784827688, 'learning_rate': 0.12322783161178008, 'max_depth': 18.0, 'num_leaves': 127.0}


In [11]:
# Train the final model with the best hyperparameters
final_gbm = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    boosting_type='gbdt',
    num_leaves=max(int(best['num_leaves']), 2),  # Ensure num_leaves is at least 2
    learning_rate=best['learning_rate'],
    feature_fraction=best['feature_fraction'],
    bagging_fraction=best['bagging_fraction'],
    bagging_freq=int(best['bagging_freq']),
    max_depth=int(best['max_depth']),
    n_estimators=100
)

In [12]:
final_gbm.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.8660324784827688, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8660324784827688
[LightGBM] [Warning] bagging_fraction is set=0.7553040682932186, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7553040682932186
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] feature_fraction is set=0.8660324784827688, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8660324784827688
[LightGBM] [Warning] bagging_fraction is set=0.7553040682932186, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7553040682932186
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000059 seconds

LGBMRegressor(bagging_fraction=0.7553040682932186, bagging_freq=2,
              feature_fraction=0.8660324784827688,
              learning_rate=0.12322783161178008, max_depth=18, metric='rmse',
              num_leaves=127, objective='regression')

In [13]:
# Save the model and label encoders
joblib.dump(final_gbm, "lgbm_model.joblib")
joblib.dump(label_encoder, "label_encoder.joblib")

['label_encoder.joblib']

## Evaluation metrics

In [14]:
# Step 10: Make predictions and evaluate the final model
y_final_pred = final_gbm.predict(X_test)
final_rmse = mean_squared_error(y_test, y_final_pred, squared=False)
final_r2 = r2_score(y_test, y_final_pred)

[LightGBM] [Warning] feature_fraction is set=0.8660324784827688, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8660324784827688
[LightGBM] [Warning] bagging_fraction is set=0.7553040682932186, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7553040682932186
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


C:\Users\TSA\anaconda3\envs\tf\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [15]:
# Print the evaluation results
print(f"Final RMSE: {final_rmse}")
print(f"Final R²: {final_r2}")

Final RMSE: 2125104.29349959
Final R²: -1.1137325707510257
